## 1. Data Preprocessing

In [ ]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# ======================
# 1. Read Data
# ======================
df1 = pd.read_csv('./Poisoning_Prediction/all_poisoning_data_wide_clean_albumin_20251106.csv')

# ======================
# 2. Define Features
# ======================
x_features_continuous = ['Age',
 'Length of Stay',
 'Weight',
 'Systolic Blood Pressure',
 'Diastolic Blood Pressure',
 'Respiratory Rate', 
 'Heart Rate',
 'White Blood Cell Count',
 'Red Blood Cell Count',
 'Hemoglobin Concentration',
 'Mean Corpuscular Volume',
 'Mean Corpuscular Hemoglobin',
 'Mean Corpuscular Hemoglobin Concentration',
 'Platelet Count',
 'Mean Platelet Volume',
 'Alanine Aminotransferase (ALT)',
 'Total Bilirubin',
 'Direct Bilirubin',
 'Lactate Dehydrogenase (LDH)',
 'Urea',
 'Serum Creatinine',
 'Uric Acid',
 'Creatine Kinase (CK)',
 'Creatine Kinase-MB Isoenzyme',
 'Troponin I',
 'High-Sensitivity C-Reactive Protein (hs-CRP)',
 'Homocysteine',
 'Potassium',
 'Sodium',
 'Chloride',
 'Carbon Dioxide',
 'Prothrombin Time',
 'D-Dimer',
 'Lactate',
 'Blood Cholinesterase Test Results',
 'Albumin (First Measurement)',
 'Albumin (Last Measurement)',
 'Number of Hemoperfusion Sessions',
 'Number of Blood Purification Sessions',
 'Hyperbaric Oxygen Therapy Duration and Frequency',
 'Atropine Dosage',
 'Long-acting Nitroglycerin Dosage',
 'Pralidoxime Dosage',
 ] 
x_features_categorical = ['Gender','Education Level','Type of Poisoning','Hypertension','Hyperlipidemia','Diabetes Mellitus','Cerebrovascular Disease','Heart Disease','Allergy History','Cancer','Poisoning','degree of poisoning','Smoking Status','Alcohol Consumption Status','Shortness of Breath','Chest Pain','Cough','Pre-syncope','Altered Consciousness or Syncope','Sore Throat','Fever','Fatigue','Lower Limb Edema','Palpitations','Vomiting','Nausea','Weakness','Headache','Residence'] # 分类变量列表


# y_column = 'Outcome_other' 
y_column = 'Outcome' 

# ======================
# 3. Shuffle Data
# ======================
df2 = df1.sample(frac=1, random_state=42).reset_index(drop=True)

# missing_summary = df2[x_features_continuous].isna().sum()
# print("原始数据含缺失值的连续特征：")
# print(missing_summary[missing_summary > 0])

In [ ]:

# Statistics Distribution of Outcome_other and Outcome

print(df2["Outcome_other"].value_counts(dropna=False))

print(df2["Outcome"].value_counts(dropna=False))


Outcome_other 分布（是否死亡）：
Outcome_other
0    889
1     82
Name: count, dtype: int64

Outcome 分布（是否未治愈）：
Outcome
0    731
1    240
Name: count, dtype: int64


In [ ]:
## Calculate the missing ratio of continuous variables

missing_ratios = df2[x_features_continuous+x_features_categorical].isnull().mean()

missing_summary = (missing_ratios * 100).round(2).sort_values(ascending=False)

print(missing_summary)

变量缺失比例（%）:
Lactate             96.81
Carbon Dioxide      94.95
Sodium              94.75
Potassium           94.75
Chloride            94.75
                    ...  
Lower Limb Edema     0.00
Vomiting             0.00
Nausea               0.00
Weakness             0.00
Headache             0.00
Length: 72, dtype: float64


In [ ]:
# Select feature names with missing rate > 90%
high_missing_features = missing_ratios[missing_ratios > 0.90].index.tolist()

for feat in high_missing_features:
    print(f"{feat}: {missing_ratios[feat]*100:.2f}%")

缺失率 > 90% 的连续变量:
Potassium: 94.75%
Sodium: 94.75%
Chloride: 94.75%
Carbon Dioxide: 94.95%
Prothrombin Time: 94.64%
D-Dimer: 94.64%
Lactate: 96.81%
Hyperbaric Oxygen Therapy Duration and Frequency: 92.38%
Atropine Dosage: 93.92%
Long-acting Nitroglycerin Dosage: 92.48%
Pralidoxime Dosage: 92.17%


In [ ]:
print(len(x_features_continuous))
print(len(high_missing_features))
x_features_continuous = [feat for feat in x_features_continuous if feat not in high_missing_features]
print(len(x_features_continuous))

df2 = df2.drop(columns=high_missing_features)
print(df2.shape)

43
11
32
(971, 95)


In [ ]:
# ======================
# 4. Fill missing values with the median of all samples
# ======================
median_values = df2[x_features_continuous].median()
df2[x_features_continuous] = df2[x_features_continuous].fillna(median_values)

missing_summary = df2[x_features_continuous].isna().sum()
print(missing_summary[missing_summary > 0])

# ======================
# 5. Normalize (standardize) each column of continuous variables
# ======================
scaler = StandardScaler()
df2[x_features_continuous] = scaler.fit_transform(df2[x_features_continuous])

check_means = df2[x_features_continuous].mean().round(3)
check_stds = df2[x_features_continuous].std().round(3)
print(pd.DataFrame({'mean': check_means.head(10), 'std': check_stds.head(10)}))


仍含缺失值的连续特征：
Series([], dtype: int64)

标准化后部分连续特征均值与标准差（应接近0与1）:
                          mean    std
Age                        0.0  1.001
Length of Stay            -0.0  1.001
Weight                     0.0  1.001
Systolic Blood Pressure   -0.0  1.001
Diastolic Blood Pressure   0.0  1.001
Respiratory Rate           0.0  1.001
Heart Rate                -0.0  1.001
White Blood Cell Count     0.0  1.001
Red Blood Cell Count      -0.0  1.001
Hemoglobin Concentration   0.0  1.001


In [ ]:

## First, fill missing values in categorical variables with "Unknown"
for col in x_features_categorical:
    if col in df2.columns:
        df2[col] = df2[col].fillna('Unknown')

# ======================
# 6. One-Hot Encoding
# ======================
x_columns = x_features_categorical + x_features_continuous
datax = df2[x_columns]
datay = df2[y_column]

datax_encoded = pd.get_dummies(datax, columns=x_features_categorical, drop_first=False)

datax_encoded = datax_encoded.astype(float)
print(f"\nOriginal number of features: {len(x_columns)}")
print(f"Number of features after One-Hot Encoding: {datax_encoded.shape[1]}")
print(f"Number of samples: {datax_encoded.shape[0]}")



原始特征数: 61
One-Hot 后特征数: 107
样本量: 971


In [ ]:
# ======================
# 7. conversion tensor
# ======================
X_tensor = torch.tensor(datax_encoded.values, dtype=torch.float32)
y_tensor = torch.tensor(datay.values, dtype=torch.float32).unsqueeze(1)

print(f"\nTensor form：X={X_tensor.shape}, y={y_tensor.shape}")


Tensor 形状：X=torch.Size([971, 107]), y=torch.Size([971, 1])


## 2.  Five-fold cross validation: Divide 1/8 of the training set into validation sets (i.e. 70% training set, 10% validation set, 20% test set)

In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score
import pandas as pd
import numpy as np
import os
import random
from sklearn import metrics

# =============== 0. Fix random seeds for reproducibility ===============
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Random seed fixed as {seed}")

set_seed(42)

# =============== 1. Output directory ===============
save_path = "./Poisoning_Prediction/DNN/predict_non-recovery_valid_test_5cv/"
os.makedirs(save_path, exist_ok=True)

# =============== 2. Device selection (CUDA / MPS / CPU) ===============
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
print(f"Using device: {device}")

# =============== 3. Convert data to tensors ===============
X_tensor = torch.tensor(datax_encoded.values, dtype=torch.float32)
y_tensor = torch.tensor(datay.values, dtype=torch.float32).unsqueeze(1)

# =============== 4. Improved DNN architecture ===============
class DNN(nn.Module):
    def __init__(self, input_dim):
        super(DNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(64, 32),
            nn.ReLU(),

            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

# =============== 5. Five-fold cross-validation ===============
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 1
auroc_list, auprc_list = [], []
all_results = []

for train_val_index, test_index in kf.split(X_tensor):
    print(f"\n===== Fold {fold} =====")
    set_seed(42 + fold)

    # Split data into train/validation/test sets
    X_train_val, X_test = X_tensor[train_val_index], X_tensor[test_index]
    y_train_val, y_test = y_tensor[train_val_index], y_tensor[test_index]
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val,
        y_train_val,
        test_size=1/8,
        random_state=42 + fold,
        stratify=y_train_val
    )

    # Compute class imbalance weight
    num_pos = (y_train == 1).sum().item()
    num_neg = (y_train == 0).sum().item()
    pos_weight = torch.tensor(num_neg / num_pos, dtype=torch.float32).to(device)
    print(f"Fold {fold}: pos_weight = {pos_weight:.2f}")

    # DataLoader with deterministic worker initialization
    def worker_init_fn(worker_id):
        np.random.seed(42 + worker_id)
        random.seed(42 + worker_id)

    train_loader = DataLoader(
        TensorDataset(X_train, y_train),
        batch_size=32,
        shuffle=True,
        num_workers=0,
        worker_init_fn=worker_init_fn
    )

    # Model, optimizer, scheduler, and loss function
    model = DNN(input_dim=X_tensor.shape[1]).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = optim.Adam(model.parameters(), lr=5e-4, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='max',
        factor=0.5,
        patience=3
    )

    # Early stopping configuration
    patience = 12
    best_auroc = 0
    wait = 0
    best_model_path = os.path.join(save_path, f"fold{fold}_best_model.pt")

    # =============== 6. Model training ===============
    max_epochs = 100
    for epoch in range(max_epochs):
        model.train()
        total_loss = 0.0

        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)

        # Validation evaluation
        model.eval()
        with torch.no_grad():
            logits = model(X_val.to(device)).squeeze()
            y_pred_prob = torch.sigmoid(logits).cpu().numpy()
            y_true = y_val.squeeze().cpu().numpy()
            auroc_val = roc_auc_score(y_true, y_pred_prob)

        scheduler.step(auroc_val)
        current_lr = optimizer.param_groups[0]['lr']

        print(
            f"Epoch {epoch+1:03d} | "
            f"Loss: {avg_loss:.4f} | "
            f"Val AUROC: {auroc_val:.4f} | "
            f"LR={current_lr:.6f}"
        )

        # Early stopping criterion
        if auroc_val > best_auroc:
            best_auroc = auroc_val
            wait = 0
            torch.save(model.state_dict(), best_model_path)
        else:
            wait += 1
            if wait >= patience:
                print(
                    f"Early stopping at epoch {epoch+1} "
                    f"(best Val AUROC={best_auroc:.4f})"
                )
                break

    # =============== 7. Test set evaluation ===============
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    model.eval()
    with torch.no_grad():
        logits = model(X_test.to(device)).squeeze()
        y_pred_prob = torch.sigmoid(logits).cpu().numpy()
        y_true = y_test.squeeze().cpu().numpy()

        auroc = roc_auc_score(y_true, y_pred_prob)
        auprc = average_precision_score(y_true, y_pred_prob)

        auroc_list.append(auroc)
        auprc_list.append(auprc)

        print(f"[Fold {fold}] Test AUROC: {auroc:.4f}, AUPRC: {auprc:.4f}")

        result_df = pd.DataFrame({
            "y_test": y_true,
            "y_pred": y_pred_prob
        })
        result_df.to_csv(
            os.path.join(save_path, f"fold{fold}_results.csv"),
            index=False
        )
        all_results.append(result_df)

    fold += 1

# =============== 8. Bootstrap estimation of overall performance ===============
def bootstrap_metric_ci(
    y_true,
    y_pred,
    metric_fn,
    n_bootstrap=2000,
    seed=42
):
    rng = np.random.RandomState(seed)
    scores = []
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    for _ in range(n_bootstrap):
        idx = rng.randint(0, len(y_true), len(y_true))
        if len(np.unique(y_true[idx])) < 2:
            continue
        scores.append(metric_fn(y_true[idx], y_pred[idx]))

    return (
        np.mean(scores),
        np.percentile(scores, 2.5),
        np.percentile(scores, 97.5)
    )

all_results_df = pd.concat(all_results, axis=0).reset_index(drop=True)
y_all_true = all_results_df["y_test"].values
y_all_pred = all_results_df["y_pred"].values

mean_auroc, auc_lower, auc_upper = bootstrap_metric_ci(
    y_all_true,
    y_all_pred,
    metrics.roc_auc_score
)
mean_auprc, auprc_lower, auprc_upper = bootstrap_metric_ci(
    y_all_true,
    y_all_pred,
    metrics.average_precision_score
)

print("\n===== Five-fold cross-validation results (Bootstrap) =====")
print(
    f"AUROC: Mean = {mean_auroc:.4f}, "
    f"95% CI = ({auc_lower:.4f}, {auc_upper:.4f})"
)
print(
    f"AUPRC: Mean = {mean_auprc:.4f}, "
    f"95% CI = ({auprc_lower:.4f}, {auprc_upper:.4f})"
)

# =============== 9. Save merged predictions from all folds ===============
all_results_path = os.path.join(save_path, "all_folds_results.csv")
all_results_df.to_csv(all_results_path, index=False)
print(f"\n✅ All fold predictions have been merged and saved to: {all_results_path}")
